### Dimenzní tabulka Country\_Date (ID, ISO, Country, Year)

In [48]:
-- Vytvoření tabulky se sloupci ID, ISO, Country, Year z tabulky Catastrophes new, protože budeme potřebovat jen záznamy       CREATE
-- uvedené v catastrophes a distinct ID, aby to byla dimenzní tabulka napojená na ostatní 
-- (vytvořeno kvůli problémům v Power BI)
SELECT
    COALESCE(c.ID, d.ID, g.ID) AS ID,
    COALESCE(c.[ISO], d.[ISO], g.[Country_code]) AS ISO,
    COALESCE(c.Country, g.Country_Name) AS Country,
    COALESCE(c.Start_Year, d.[Year], g.[Year]) AS Year
INTO Country_Date
FROM (
    SELECT DISTINCT ID FROM [Catastrophes new]
    UNION
    SELECT DISTINCT ID FROM [Direct_disaster_economic_loss_modified]
    UNION
    SELECT DISTINCT ID FROM [Gross_Domestic_Product]
) AS ids
LEFT JOIN [Catastrophes new] c ON ids.ID = c.ID
LEFT JOIN [Direct_disaster_economic_loss_modified] d ON ids.ID = d.ID
LEFT JOIN [Gross_Domestic_Product] g ON ids.ID = g.ID;

(30930 rows affected)

Total execution time: 00:00:03.882

In [51]:
--Smazání duplicitních řádků ID                                                                                               CREATE
WITH RankedCountryDate AS (
    SELECT
        ID,
        ISO,
        Country,
        Year,
        ROW_NUMBER() OVER (PARTITION BY ID ORDER BY ID) AS RowNum
    FROM Country_Date
)

DELETE FROM RankedCountryDate WHERE RowNum > 1;

(19061 rows affected)

Total execution time: 00:00:02.612